<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5dd07cb9749ec55e370a4953040c9e8d09f0be2f41406a2718634ce7310fb9f1
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-10 09:42:45
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  57.44 K
Current:  1.42 C
-------------------
Today PnL: 50.01 K (0.35%)
Current PnL: -19.90 L (-13.26%)
CY Booked + Current PnL: -8.46 L (-5.64%)
-------------------
Total profit:  1.71 L
Total loss:  -21.61 L
-------------------
Total Booked + Current PnL: 18.52 L (15.04%)
Total Booked PnL: 38.42 L (31.2%)
Curr Year Booked PnL: 11.44 L (8.07%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 87.41 L (61.69%)
Deployed:  1.23 C
Current:  1.42 C
CAGR/XIRR %: 8.45%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,66.0,H-MC,14.30,119520.0,19016.0,10087.0,-1.76,18.92,8.44,28.96,79.0,1.89,0.85,59.31,MH,ATH,METALS
14,BLUESTARCO,2080.00,11.47,55.0,X-MC,3.16,192500.0,27830.0,15496.0,1.73,16.90,8.05,26.31,89.0,1.80,1.36,26.23,X40N,NTT,AC
77,TTKPRESTIG,770.00,93.60,40.0,M-SC,0.99,83447.0,-17330.0,17424.0,-0.01,-17.20,20.88,0.09,245.0,-0.99,0.59,9.63,OX40N,NTT,DURABLES
84,VOLTAS,1530.00,1.02,61.0,X-MC,1.08,211980.0,20238.0,17510.0,2.26,10.55,8.26,19.69,99.0,1.16,1.50,18.07,XY25,NTT,AC
19,CIPLA,1662.74,-22.18,46.0,X-LC,3.61,207706.0,3206.0,20085.0,0.20,1.57,9.67,11.39,10.0,0.16,1.47,10.66,X40N,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,139.18,69.0,H-SC,7.15,162094.0,-56402.0,84597.0,12.86,-25.81,52.19,12.90,153.0,-0.67,1.15,23.39,OX40N,NTT,FINANCE
67,SONACOMS,1006.00,-36.95,61.0,M-SC,13.28,78391.0,-22768.0,97660.0,4.31,-22.51,124.58,74.03,202.0,-0.23,0.56,10.54,AR,BTT,AUTO
46,KPIGREEN,731.05,-7.49,37.0,H-SC,6.26,110023.0,-15274.0,74200.0,3.51,-12.19,67.44,47.03,141.0,-0.21,0.78,37.76,MH,ATH,POWER
85,WHIRLPOOL,2270.00,-47.99,33.0,M-SC,2.41,87968.0,-3530.0,82285.0,2.52,-3.86,93.54,86.07,223.0,-0.04,0.62,27.74,XR,NTT,DURABLES
84,VOLTAS,1530.00,1.02,61.0,X-MC,1.08,211980.0,20238.0,17510.0,2.26,10.55,8.26,19.69,99.0,1.16,1.50,18.07,XY25,NTT,AC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,31.71,62.0,M-LC,7.74,214781.0,9705.0,102859.0,-3.61,4.73,47.89,54.89,52.0,0.09,1.52,29.97,X5K,ATH,METALS
60,SAIL,228.00,45.95,53.0,M-MC,12.72,229887.0,4925.0,162737.0,-2.17,2.19,70.79,74.53,192.0,0.03,1.63,35.15,XY24,BTT,STEEL
72,TATAELXSI,9161.00,-23.37,48.0,H-MC,6.62,103664.0,-28171.0,70398.0,-2.10,-21.37,67.91,32.03,98.0,-0.40,0.73,16.11,OX40N,NTT,IT
50,NATIONALUM,244.55,-40.35,66.0,H-MC,14.30,119520.0,19016.0,10087.0,-1.76,18.92,8.44,28.96,79.0,1.89,0.85,59.31,MH,ATH,METALS
82,VALIANTORG,838.00,-292.76,31.0,H-SC,1.66,107134.0,-56471.0,177789.0,-1.27,-34.52,165.95,74.15,139.0,-0.32,0.76,37.33,XR,NTT,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,105.37,61.0,H-SC,13.72,129978.0,-12501.0,32027.0,-0.15,-8.77,24.64,13.70,163.0,-0.39,0.92,53.41,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.60,40.0,M-SC,0.99,83447.0,-17330.0,17424.0,-0.01,-17.20,20.88,0.09,245.0,-0.99,0.59,9.63,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,49.0,H-SC,1.93,221220.0,-48447.0,84772.0,0.18,-17.97,38.32,13.47,138.0,-0.57,1.57,12.48,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-23.37,48.0,H-MC,6.62,103664.0,-28171.0,70398.0,-2.10,-21.37,67.91,32.03,98.0,-0.40,0.73,16.11,OX40N,NTT,IT
18,CERA,9475.0,-24.58,33.0,H-SC,1.81,137298.0,-38605.0,80621.0,-0.89,-21.95,58.72,23.89,149.0,-0.48,0.97,18.58,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.95,53.0,M-MC,12.72,229887.0,4925.0,162737.0,-2.17,2.19,70.79,74.53,192.0,0.03,1.63,35.15,XY24,BTT,STEEL
32,HINDZINC,730.22,31.71,62.0,M-LC,7.74,214781.0,9705.0,102859.0,-3.61,4.73,47.89,54.89,52.0,0.09,1.52,29.97,X5K,ATH,METALS
86,WIPRO,420.00,-12.43,54.0,M-LC,5.73,153760.0,2815.0,106633.0,0.65,1.86,69.35,72.51,53.0,0.03,1.09,8.00,XR,NTT,IT
79,UNIONBANK,163.00,-13.18,56.0,M-LC,9.15,157440.0,16600.0,27898.0,0.46,11.79,17.72,31.59,66.0,0.60,1.12,41.40,XY24,NTT,BANKS
11,BANKINDIA,190.00,-24.22,67.0,H-MC,11.91,194034.0,14226.0,98181.0,0.29,7.91,50.60,62.52,88.0,0.14,1.38,44.70,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,66.0,H-MC,14.30,119520.0,19016.0,10087.0,-1.76,18.92,8.44,28.96,79.0,1.89,0.85,59.31,MH,ATH,METALS
11,BANKINDIA,190.00,-24.22,67.0,H-MC,11.91,194034.0,14226.0,98181.0,0.29,7.91,50.60,62.52,88.0,0.14,1.38,44.70,XR,NTT,BANKS
32,HINDZINC,730.22,31.71,62.0,M-LC,7.74,214781.0,9705.0,102859.0,-3.61,4.73,47.89,54.89,52.0,0.09,1.52,29.97,X5K,ATH,METALS
37,INDIAMART,4810.62,-54.45,43.0,H-SC,8.74,127115.0,3779.0,127852.0,-0.57,3.06,100.58,106.72,119.0,0.03,0.90,27.16,AR,ATH,MISC
86,WIPRO,420.00,-12.43,54.0,M-LC,5.73,153760.0,2815.0,106633.0,0.65,1.86,69.35,72.51,53.0,0.03,1.09,8.00,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-46.56,32.0,X-SC,37.79,52987.0,-12019.0,162946.0,1.66,-18.49,307.52,232.18,198.0,-0.07,0.38,1.66,XY24,NTT,MISC
5,ASIANPAINT,3465.66,-18.03,36.0,X-LC,5.55,206747.0,-45021.0,98225.0,0.56,-17.88,47.51,21.13,27.0,-0.46,1.47,11.21,X40,ATH,PAINTS
23,DABUR,735.00,-12.30,36.0,X-MC,3.75,237480.0,-10588.0,119001.0,0.88,-4.27,50.11,43.70,102.0,-0.09,1.68,9.56,XY24,BTT,FMCG
22,COLPAL,3767.14,-5.20,36.0,X-MC,7.29,221200.0,-42165.0,155504.0,0.15,-16.01,70.30,43.04,84.0,-0.27,1.57,2.69,XY25,ATH,FMCG
29,HAVELLS,2069.16,-6.18,37.0,X-MC,0.58,232612.0,-15119.0,90184.0,0.36,-6.10,38.77,30.30,92.0,-0.17,1.65,6.38,X40,ATH,ELECTRICAL


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,HAVELLS,2069.16,-6.18,37.0,X-MC,0.58,232612.0,-15119.0,90184.0,0.36,-6.10,38.77,30.30,92.0,-0.17,1.65,6.38,X40,ATH,ELECTRICAL
84,VOLTAS,1530.00,1.02,61.0,X-MC,1.08,211980.0,20238.0,17510.0,2.26,10.55,8.26,19.69,99.0,1.16,1.50,18.07,XY25,NTT,AC
73,TATAMOTORS,1065.00,-21.50,45.0,X-LC,1.60,247602.0,-28921.0,139004.0,0.15,-10.46,56.14,39.81,54.0,-0.21,1.75,18.67,XY24,NTT,AUTO
65,SIEMENS,4671.50,-0.68,51.0,H-LC,1.70,159720.0,-26375.0,73855.0,0.50,-14.17,46.24,25.51,15.0,-0.36,1.13,17.70,AR,ATH,ELECTRICAL
34,ICICIGI,2260.25,-18.49,51.0,X-MC,1.88,140156.0,4023.0,27106.0,0.60,2.95,19.34,22.86,91.0,0.15,0.99,15.94,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-46.56,32.0,X-SC,37.79,52987.0,-12019.0,162946.0,1.66,-18.49,307.52,232.18,198.0,-0.07,0.38,1.66,XY24,NTT,MISC
22,COLPAL,3767.14,-5.20,36.0,X-MC,7.29,221200.0,-42165.0,155504.0,0.15,-16.01,70.30,43.04,84.0,-0.27,1.57,2.69,XY25,ATH,FMCG
83,VBL,671.64,-20.64,42.0,X-LC,10.32,285920.0,-29922.0,142588.0,0.50,-9.47,49.87,35.67,5.0,-0.21,2.03,3.07,X40N,ATH,FMCG
42,ITC,452.00,-39.19,45.0,X-LC,2.41,196578.0,-3560.0,24454.0,0.53,-1.78,12.44,10.44,4.0,-0.15,1.39,3.69,X40,NTT,FMCG
9,BAJAJHFL,181.50,-14.27,40.0,X-MC,4.95,180728.0,-19778.0,116931.0,0.10,-9.86,64.70,48.45,90.0,-0.17,1.28,4.22,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-46.56,32.0,X-SC,37.79,52987.0,-12019.0,162946.0,1.66,-18.49,307.52,232.18,198.0,-0.07,0.38,1.66,XY24,NTT,MISC
56,RELAXO,1176.00,-39.97,38.0,X-SC,2.54,81874.0,-63286.0,139210.0,-0.55,-43.60,170.03,52.31,136.0,-0.45,0.58,9.59,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-34.44,46.0,X-SC,4.41,98048.0,-30622.0,75919.0,0.77,-23.80,77.43,35.20,219.0,-0.40,0.69,12.54,X40,NTT,FOOTWEAR
17,CAMS,5211.76,-4.72,52.0,X-SC,1.44,108318.0,6314.0,37608.0,0.98,6.19,34.72,43.06,122.0,0.17,0.77,24.37,X40N,ATH,MISC
33,HONAUT,58357.33,-22.51,48.0,X-SC,2.13,109245.0,-18693.0,65831.0,0.48,-14.61,60.26,36.84,143.0,-0.28,0.77,11.84,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-28.36,51.0,X-LC,12.15,285187.0,-60769.0,129019.0,-0.91,-17.57,45.24,19.73,1.0,-0.47,2.02,5.04,X40,ATH,IT
40,INFY,2275.00,-17.44,57.0,X-LC,7.78,321843.0,8955.0,162724.0,0.11,2.86,50.56,54.87,3.0,0.06,2.28,9.70,X40,BTT,IT
42,ITC,452.00,-39.19,45.0,X-LC,2.41,196578.0,-3560.0,24454.0,0.53,-1.78,12.44,10.44,4.0,-0.15,1.39,3.69,X40,NTT,FMCG
83,VBL,671.64,-20.64,42.0,X-LC,10.32,285920.0,-29922.0,142588.0,0.50,-9.47,49.87,35.67,5.0,-0.21,2.03,3.07,X40N,ATH,FMCG
1,ABB,7934.00,-40.35,48.0,H-LC,7.49,249408.0,-12211.0,131413.0,0.96,-4.67,52.69,45.57,7.0,-0.09,1.77,6.13,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,50.0,L-SC,29.05,81312.0,-32237.0,72286.0,-0.82,-28.39,88.90,35.27,268.0,-0.45,0.58,99.01,XR,NTT,HOTELS
49,MASFIN,398.61,-20.51,40.0,H-SC,10.35,90330.0,-7650.0,29249.0,-0.03,-7.81,32.38,22.05,152.0,-0.26,0.64,31.30,XR,ATH,FINANCE
50,NATIONALUM,244.55,-40.35,66.0,H-MC,14.30,119520.0,19016.0,10087.0,-1.76,18.92,8.44,28.96,79.0,1.89,0.85,59.31,MH,ATH,METALS
78,UJJIVANSFB,60.00,105.37,61.0,H-SC,13.72,129978.0,-12501.0,32027.0,-0.15,-8.77,24.64,13.70,163.0,-0.39,0.92,53.41,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-182.95,62.0,M-SC,15.27,143388.0,-5832.0,150013.0,-1.08,-3.91,104.62,96.62,208.0,-0.04,1.02,55.18,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,64.47,64.0,H-SC,8.97,151613.0,-27458.0,93181.0,0.03,-15.33,61.46,36.71,135.0,-0.29,1.07,50.43,XR,NTT,BANKS
61,SAMMAANCAP,326.00,-182.95,62.0,M-SC,15.27,143388.0,-5832.0,150013.0,-1.08,-3.91,104.62,96.62,208.0,-0.04,1.02,55.18,XY25,NTT,FINANCE
32,HINDZINC,730.22,31.71,62.0,M-LC,7.74,214781.0,9705.0,102859.0,-3.61,4.73,47.89,54.89,52.0,0.09,1.52,29.97,X5K,ATH,METALS
26,GREENPANEL,537.00,232.04,58.0,M-SC,4.66,148303.0,-34775.0,113748.0,-0.38,-18.99,76.70,43.14,230.0,-0.31,1.05,39.50,XY24,NTT,MISC
50,NATIONALUM,244.55,-40.35,66.0,H-MC,14.30,119520.0,19016.0,10087.0,-1.76,18.92,8.44,28.96,79.0,1.89,0.85,59.31,MH,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.15
1,25,43.40
2,50,75.18


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.04
LC    32.24
MC    23.71
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.90
X40      14.61
X40N     11.23
XR       10.92
XY25     10.39
AR        8.21
OX40N     7.74
X200      1.73
MH        1.63
X5K       1.52
SR        1.11
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.58
X-LC    22.87
X-MC    17.18
M-SC    12.51
H-LC     4.63
H-MC     4.60
X-SC     4.52
M-LC     3.73
M-MC     1.63
L-SC     1.43
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.75,-5.80,40.67
IT,12.83,-16.10,78.98
FINANCE,10.62,-12.67,67.09
MISC,6.90,-18.05,83.46
BANKS,6.34,-11.48,70.13
PAINTS,5.50,-19.20,36.97
ELECTRICAL,5.32,-9.64,48.11
INSURANCE,3.80,-3.14,41.69
AC,3.55,3.45,12.71


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3212083.0,22
XR,1367839.0,14
AR,1171799.0,9
X40,777462.0,10
X40N,651338.0,9
OX40N,553764.0,10
XY25,472029.0,7
SR,265399.0,2
X5K,102859.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3389244.0,25
M-SC,1275951.0,15
X-MC,1109927.0,12
X-LC,1013682.0,13
X-SC,543168.0,6
H-MC,360378.0,4
H-LC,286948.0,3
L-SC,259495.0,3
M-LC,237390.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1195816.0      6
           AR         804233.0      5
           XR         801121.0      7
M-SC       XY24       686228.0      6
X-MC       XY24       588094.0      4
X-LC       X40        485151.0      5
X-SC       X40N       304303.0      4
H-SC       SR         265399.0      2
           OX40N      248475.0      4
X-MC       X40        216392.0      4
X-LC       X40N       214608.0      3
           XY24       206652.0      2
H-LC       AR         205268.0      2
H-MC       XY24       181712.0      1
X-MC       XY25       173014.0      2
L-SC       XR         170827.0      2
X-SC       XY24       162946.0      1
M-MC       XY24       162737.0      1
M-SC       AR         162298.0      2
           XY25       150013.0      1
           OX40N      146223.0      4
X-MC       X40N       132427.0      2
M-SC       XR         131189.0      2
X-LC       XY25       107271.0      3
M-LC       XR         106633.0      1
           X5K        102859.0      1
H-MC       XR          98181.0      1
L-SC       OX40N       88668.0      1
H-LC       X200        81680.0      1
X-SC       X40         75919.0      1
H-SC       MH          74200.0      1
H-MC       OX40N       70398.0      1
L-MC       XR          59888.0      1
L-LC       XY25        41731.0      1
M-LC       XY24        27898.0      1
H-MC       MH          10087.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
